# Task 1
### Task 1.1
##### Write a function that computes the total buy volume for flex power, another that computes the total sell volume.

In [1]:
import sqlite3 # Import sqlite3 to acess the database in python

def compute_total_buy_volume(db_path: str = 'trades.sqlite') -> float:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Sum up the quantities (volume) on the buyside
    cursor.execute('''
        SELECT SUM(quantity) FROM epex_12_20_12_13
        WHERE side = 'buy'
    ''')
    res = cursor.fetchone()[0]
    conn.close()

    return res if res else 0.0

# Identical procedure on the sell-side
def compute_total_sell_volume(db_path: str = 'trades.sqlite') -> float:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute('''
        SELECT SUM(quantity) FROM epex_12_20_12_13
        WHERE side = 'sell'
    ''')
    res = cursor.fetchone()[0]
    conn.close()

    return res if res else 0.0

In [2]:
buy_volume = compute_total_buy_volume()
sell_volume = compute_total_sell_volume()

print(f'Total Buy Volume: {buy_volume} MW')
print(f'Total Sell Volume: {sell_volume} MW')


Total Buy Volume: 28 MW
Total Sell Volume: 53 MW


### Task 1.2
##### Write a function that computes the PnL (profit and loss) of each strategy in euros. It's defined as the sum of the incomes realized with each trade. If we sell energy, our income is quantity * price since we got money for our electricity. If we buy energy, our income is -quantity * price.

In [25]:
'''
The idea is very similar to Task 1.1: By acessing the sell-side of the specific strategy_id and
multiplying the quantity with price, we get the amount we receive by selling positions.
On the other hand we get the buying cost by acessing the buy-side. 
Deducting the cost from the income we get the P&L of the strategy.
'''

def compute_pnl(strategy_id: str, db_path: str = 'trades.sqlite') -> float:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Sell income: +quantity * price
    cursor.execute('''
        SELECT SUM(quantity * price) FROM epex_12_20_12_13
        WHERE strategy = ? AND side = 'sell'
    ''', (strategy_id,))
    sell_income = cursor.fetchone()[0] or 0.0

    # Buy income: -quantity * price
    cursor.execute('''
        SELECT SUM(quantity * -price) FROM epex_12_20_12_13
        WHERE strategy = ? AND side = 'buy'
    ''', (strategy_id,))
    buy_cost = cursor.fetchone()[0] or 0.0

    conn.close()

    pnl = sell_income + buy_cost
    return pnl


In [26]:
# Testing the function with different Strategies
print(f'The P&L of strategy_1 is {compute_pnl("strategy_1"):.2f}€')
print(f'The P&L of strategy_2 is {compute_pnl("strategy_2"):.2f}€')
print(f'The P&L of strategy_3 is {compute_pnl("strategy_3"):.2f}€') # should be zero since no 'strategy_3' exists


The P&L of strategy_1 is -50.00€
The P&L of strategy_2 is 548.00€
The P&L of strategy_3 is 0.00€


### Task 1.3
##### Expose the function defined in the second task as an entrypoint of a web application.


In [12]:
from flask import Flask, jsonify
from datetime import datetime
import sqlite3
from threading import Thread

app = Flask(__name__)


@app.route('/v1/pnl/<strategy_id>', methods=['GET'])
def get_pnl(strategy_id):
    pnl = compute_pnl(strategy_id)
    response = {
        'strategy': strategy_id,
        'value': round(pnl, 2),
        'unit': 'euro',
        'capture_time': datetime.utcnow().isoformat() + 'Z'
    }
    return jsonify(response), 200

# Run Flask 
def run_app():
    app.run(port=5000)

thread = Thread(target=run_app)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2025 11:24:18] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [13/Apr/2025 11:24:34] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [13/Apr/2025 11:24:48] "GET /v1/pnl/YOUR_STRATEGY_ID HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2025 11:26:20] "GET /v1/pnl/strategy_1 HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2025 11:26:28] "GET /v1/pnl/strategy_2 HTTP/1.1" 200 -
